In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
!pwd
% cd /content/drive/MyDrive/your_path_to_the_dir
!ls

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import Trainer

In [ ]:
#Load data
campus_dataset = load_dataset("csv", data_files="enhanced_questions_int.csv")

#train and test
campus_dataset = campus_dataset["train"].train_test_split(train_size=0.9, seed=45)

In [ ]:
campus_dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'text'],
        num_rows: 524
    })
    test: Dataset({
        features: ['labels', 'text'],
        num_rows: 59
    })
})

In [ ]:
#Save in arrow format

campus_dataset.save_to_disk('campus_questions_dataset')

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-5f3e9ec34791bff8/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a/cache-875dc06d58f49fcd.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-5f3e9ec34791bff8/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a/cache-bb37941d3a865b65.arrow


# Creating checkpoints

In [ ]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.5",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/

In [ ]:
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)


tokenized_datasets = campus_dataset.map(tokenize_function, batched=True)
tokenized_datasets.set_format("torch")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

#tokenized_datasets = tokenized_datasets.rename_column('labels', 'labels')
tokenized_datasets = tokenized_datasets.remove_columns(["text"])

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-5f3e9ec34791bff8/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a/cache-1d3a11cdd2c42964.arrow


  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets['train']


Dataset({
    features: ['attention_mask', 'input_ids', 'labels', 'token_type_ids'],
    num_rows: 524
})

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["test"], batch_size=8, collate_fn=data_collator
)

# Metrics

In [ ]:
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  acc = accuracy_score(labels, preds)
  return {
#      'accuracy': acc,
  }

# Training


In [ ]:
training_args = TrainingArguments("test-trainer")

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=20,   # batch size for evaluation
    warmup_steps=100,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=50,               # log & save weights each logging_steps
    save_steps=500,
    evaluation_strategy="steps",     # evaluate each `logging_steps`
)

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=6)

training_args

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
using `logging_steps` to initialize `eval_steps` to 50
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "a

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_steps=50,
evaluation_strategy=IntervalStrategy.STEPS,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=False,
group_by_length=False,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_at_end=True,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_node=True,
logging_dir=./logs,
logging_first_step=False,
logging_nan_inf_filter=True,
logging_steps=50,
logging_strategy=IntervalStr

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics = compute_metrics,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 524
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 330


Step,Training Loss,Validation Loss
50,1.631000,1.315421
100,0.772600,0.184475
150,0.074700,0.012884
200,0.011000,0.005385
250,0.004800,0.003323
300,0.003900,0.002880


***** Running Evaluation *****
  Num examples = 59
  Batch size = 20
***** Running Evaluation *****
  Num examples = 59
  Batch size = 20
***** Running Evaluation *****
  Num examples = 59
  Batch size = 20
***** Running Evaluation *****
  Num examples = 59
  Batch size = 20
***** Running Evaluation *****
  Num examples = 59
  Batch size = 20
***** Running Evaluation *****
  Num examples = 59
  Batch size = 20


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=330, training_loss=0.37878330887267087, metrics={'train_runtime': 26.75, 'train_samples_per_second': 97.944, 'train_steps_per_second': 12.336, 'total_flos': 17039108316816.0, 'train_loss': 0.37878330887267087, 'epoch': 5.0})

# Evaluation

In [ ]:
trainer.evaluate()



***** Running Evaluation *****
  Num examples = 59
  Batch size = 20


{'epoch': 5.0,
 'eval_loss': 0.0028184857219457626,
 'eval_runtime': 0.1039,
 'eval_samples_per_second': 567.976,
 'eval_steps_per_second': 28.88}

In [ ]:
#save the model

model_path = "botcamp-bert-base-uncased"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

Configuration saved in botcamp-bert-base-uncased/config.json
Model weights saved in botcamp-bert-base-uncased/pytorch_model.bin
tokenizer config file saved in botcamp-bert-base-uncased/tokenizer_config.json
Special tokens file saved in botcamp-bert-base-uncased/special_tokens_map.json


('botcamp-bert-base-uncased/tokenizer_config.json',
 'botcamp-bert-base-uncased/special_tokens_map.json',
 'botcamp-bert-base-uncased/vocab.txt',
 'botcamp-bert-base-uncased/added_tokens.json',
 'botcamp-bert-base-uncased/tokenizer.json')

In [ ]:
def get_prediction(text):
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt").to("cuda")
    # perform inference to our model
    outputs = model(**inputs)
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1)
    # executing argmax function to get the candidate label
    return probs, probs.argmax()

In [ ]:
text = "I want to eat. "
print(get_prediction(text))
t, p = get_prediction(text)
p.item()

(tensor([[2.5115e-04, 3.4493e-04, 9.9836e-01, 3.8031e-04, 3.9956e-04, 2.6372e-04]],
       device='cuda:0', grad_fn=<SoftmaxBackward0>), tensor(2, device='cuda:0'))


2